In [2]:
import numpy as np

In [8]:
# First, because I'm missing actual input data, lets find a way to generate some.
# I will use player number as their skill, just because it's easy. I don't want it completely
# random because then what is the model learning, really?
# Also lets have more players than actually appear in a single game
players = np.arange(20) + 1
maps = ["The Skeld", "MIRA HQ", "Polus", "Airship"]
no_crewmates = 8
no_impostors = 2
confirm_ejects = False
emergency_meetings = np.arange(10)
emergency_cooldown = np.arange(0, 61, 5)
discussion_time = np.arange(0, 121, 15)
voting_time = np.arange(0, 301, 15)
anonymous_votes = True
player_speed = 1.25
crewmate_vision = np.arange(0, 5, 0.25) + 0.25
impostor_vision = np.arange(0, 5, 0.25) + 0.25
kill_cooldown = np.arange(10, 61, 2.5)
kill_distance = ["Short", 'Medium', "Long"]
visual_tasks = False
task_bar_updates = ["Always", "Meetings", "Never"]
common_tasks = np.arange(3)
long_tasks = np.arange(4)
short_tasks = np.arange(6)

In [79]:
rng = np.random.default_rng()
def get_winner(crewmates, impostors, settings):
    impostor_skill = np.sum(impostors) * 4
    crewmate_skill = np.sum(crewmates)
    
    # Incoporate game settings
    # Don't include common because they can also be used against impostors
    crewmate_skill /= (settings['short_tasks'] + settings['long_tasks'])
    crewmate_skill *= min(settings['crewmate_vision'], 2)
    crewmate_skill *= max(settings['discussion_time'] // 25, 1)
    crewmate_skill *= (settings['voting_time'] + 5) // 25
    if settings['anonymous_votes']:
        crewmate_skill *= 0.85
    if settings['confirm_ejects']:
        crewmate_skill *= 4
    crewmate_skill *= max(
        (settings['kill_cooldown'] - settings['emergency_cooldown']) / 15,
        0.25
    )
    crewmate_skill *= settings['emergency_meetings'] / 25 + 1
    
    impostor_skill *= min(settings['impostor_vision'], 1.25)
    kill_mod = { 'Short':1, 'Medium':2, 'Long':3 }
    impostor_skill *= kill_mod[settings['kill_distance']]
    taskbar = { 'Always':1, 'Meetings':2, 'Never':3 }
    impostor_skill *= taskbar[settings['task_bar_updates']]
    if settings['visual_tasks']:
        impostor_skill /= 4
    impostor_skill /= settings['kill_cooldown'] / 30
    
    impostor_skill = np.ceil(impostor_skill)
    crewmate_skill = np.ceil(crewmate_skill)
    match_value = rng.integers(impostor_skill + crewmate_skill)
    return 'crewmates' if match_value < crewmate_skill else 'impostors'

def generate_settings():
    settings = {
        'map': rng.choice(maps),
        'confirm_ejects': bool(rng.integers(2)),
        'emergency_meetings': int(rng.choice(emergency_meetings)),
        'emergency_cooldown': int(rng.choice(emergency_cooldown)),
        'discussion_time': int(rng.choice(discussion_time)),
        'voting_time': int(rng.choice(voting_time)),
        'anonymous_votes': bool(rng.integers(2)),
        'player_speed': float(player_speed),
        'crewmate_vision': float(rng.choice(crewmate_vision)),
        'impostor_vision': float(rng.choice(impostor_vision)),
        'kill_cooldown': float(rng.choice(kill_cooldown)),
        'kill_distance': rng.choice(kill_distance),
        'visual_tasks': bool(rng.integers(2)),
        'task_bar_updates': rng.choice(task_bar_updates),
        'common_tasks': int(rng.choice(common_tasks)),
        'long_tasks': int(rng.choice(long_tasks)),
        'short_tasks': int(rng.choice(short_tasks))
    }
    if settings['long_tasks'] + settings['short_tasks'] == 0:
        settings['short_tasks'] = 1
    return settings

In [68]:
wins = {
    'crewmates': 0,
    'impostors': 0
}
for _ in range(1000):
    wins[get_winner(10, 3, generate_settings())] += 1

print(wins)
#print(skillrange)

{'crewmates': 566, 'impostors': 434}
{'crew': [0.0, 6253.0], 'imp': [1.0, 405.0]}


I'm happy enough now with the win ratio between random settings, without taking players into account. Now lets add players

In [80]:
def generate_match():
    impostors = rng.choice(players, no_impostors, replace=False)
    crewmates = np.setdiff1d(players, impostors, assume_unique=True)
    crewmates = rng.choice(crewmates, no_crewmates, replace=False)
    
    settings = generate_settings()
    winner = get_winner(crewmates, impostors, settings)
    
    stradd = np.core.defchararray.add
    crew_names = list(stradd('Player ', crewmates.astype(str)))
    imp_names = list(stradd('Player ', impostors.astype(str)))
    
    match_result = {
        'crewmates': crew_names,
        'impostors': imp_names,
        **settings,
        'match_winner': winner
    }
    return match_result
    
generate_match()

{'crewmates': ['Player 18',
  'Player 7',
  'Player 6',
  'Player 16',
  'Player 4',
  'Player 20',
  'Player 13',
  'Player 10'],
 'impostors': ['Player 19', 'Player 8'],
 'map': 'Airship',
 'confirm_ejects': False,
 'emergency_meetings': 1,
 'emergency_cooldown': 35,
 'discussion_time': 75,
 'voting_time': 105,
 'anonymous_votes': False,
 'player_speed': 1.25,
 'crewmate_vision': 0.25,
 'impostor_vision': 2.25,
 'kill_cooldown': 12.5,
 'kill_distance': 'Medium',
 'visual_tasks': False,
 'task_bar_updates': 'Never',
 'common_tasks': 0,
 'long_tasks': 1,
 'short_tasks': 4,
 'match_winner': 'impostors'}

Just visually checking a bunch of generated matches, the winning team seems actually really quite reasonable. Lets generate a dataset

In [81]:
match_count = 10000
matches = [generate_match() for _ in range(match_count)]
pwins = {f'Player {p}': 0 for p in players}
twins = {
    'crewmates': 0,
    'impostors': 0
}
for match in matches:
    twins[match['match_winner']] += 1
    # ngl this is a bit of a happy accident
    # I wasn't intending to set it up so I could access the winning
    # players like this so easily
    for p in match[match['match_winner']]:
        pwins[p] += 1
        
print(pwins)
print(twins)

{'Player 1': 2708, 'Player 2': 2704, 'Player 3': 2719, 'Player 4': 2744, 'Player 5': 2709, 'Player 6': 2725, 'Player 7': 2797, 'Player 8': 2821, 'Player 9': 2830, 'Player 10': 2784, 'Player 11': 2805, 'Player 12': 2854, 'Player 13': 2850, 'Player 14': 2831, 'Player 15': 2908, 'Player 16': 2949, 'Player 17': 2911, 'Player 18': 2893, 'Player 19': 2896, 'Player 20': 2982}
{'crewmates': 6070, 'impostors': 3930}


In [82]:
matches[0]

{'crewmates': ['Player 3',
  'Player 9',
  'Player 17',
  'Player 16',
  'Player 6',
  'Player 14',
  'Player 1',
  'Player 4'],
 'impostors': ['Player 2', 'Player 5'],
 'map': 'MIRA HQ',
 'confirm_ejects': False,
 'emergency_meetings': 3,
 'emergency_cooldown': 30,
 'discussion_time': 15,
 'voting_time': 75,
 'anonymous_votes': True,
 'player_speed': 1.25,
 'crewmate_vision': 4.5,
 'impostor_vision': 4.5,
 'kill_cooldown': 25.0,
 'kill_distance': 'Medium',
 'visual_tasks': True,
 'task_bar_updates': 'Always',
 'common_tasks': 2,
 'long_tasks': 1,
 'short_tasks': 1,
 'match_winner': 'crewmates'}

In [83]:
import json
with open('data/matches.json', 'w') as outfile:
    json.dump(matches, outfile)